In [1]:
import os
import copy
import json
import logging

import torch
from torch.utils.data import Dataset

from datasets import Dataset as HFDataset  # Lưu ý phân biệt Dataset của HuggingFace với torch.utils.data
from sklearn.metrics import classification_report

from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    Trainer,
    TrainingArguments,
    EvalPrediction
)

In [2]:
def read_conll(file_path):
    sentences = []
    sentence_labels = []
    unique_labels = set()

    with open(file_path, 'r', encoding='utf-8') as file:
        current_sentence_tokens = []
        current_sentence_labels = []

        for line in file:
            line = line.strip()

            # Nếu gặp dòng trống => kết thúc 1 câu
            if not line:
                if current_sentence_tokens:
                    sentences.append(' '.join(current_sentence_tokens))
                    sentence_labels.append(' '.join(current_sentence_labels))
                current_sentence_tokens = []
                current_sentence_labels = []
            else:
                line_parts = line.split()
                current_sentence_tokens.append(line_parts[0])

                if len(line_parts) >= 4:
                    current_sentence_labels.append(line_parts[3])
                    unique_labels.add(line_parts[3])
                else:
                    current_sentence_labels.append('O')

    # Trường hợp file không kết thúc bằng dòng trống
    if current_sentence_tokens:
        sentences.append(' '.join(current_sentence_tokens))
        sentence_labels.append(' '.join(current_sentence_labels))

    return sentences, sentence_labels


def read_conll_directory(directory_path):
    all_sentences = []
    all_sentence_labels = []

    # Duyệt qua các file trong thư mục
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.conll'):
            file_path = os.path.join(directory_path, file_name)
            print(f"Processing file: {file_path}")

            sentences, sentence_labels = read_conll(file_path)
            all_sentences.extend(sentences)
            all_sentence_labels.extend(sentence_labels)

    return all_sentences, all_sentence_labels


def extract_labels(label_sentences):
    """
    Hàm để lấy danh sách nhãn từ danh sách các câu nhãn.
    """
    unique_labels = set()
    for sentence in label_sentences:
        labels = sentence.split()
        unique_labels.update(labels)
    return unique_labels


def prepare_dataset(sentences, labels):
    """
    Chuẩn bị dữ liệu ở dạng dictionary gồm 2 trường: tokens và labels (cùng độ dài).
    """
    return {'tokens': sentences, 'labels': labels}


def process_string_to_array(dataset):
    """
    Chuyển chuỗi tokens/labels thành list (array) tokens/labels để đưa vào mô hình.
    """
    return {
        'tokens': [sentence.split() for sentence in dataset['tokens']],
        'labels': [label_seq.split() for label_seq in dataset['labels']]
    }

In [3]:
train_directory = '/kaggle/input/vlsp2021ner/vlspner2021/Train-Conll'
dev_directory = '/kaggle/input/vlsp2021ner/vlspner2021/Dev-Conll'

train_sentences, train_labels = read_conll_directory(train_directory)
dev_sentences, dev_labels = read_conll_directory(dev_directory)

# Xây dựng tập dữ liệu Train/Dev dưới dạng dict
train_dataset_dict = prepare_dataset(train_sentences, train_labels)
dev_dataset_dict = prepare_dataset(dev_sentences, dev_labels)

# Chuyển tokens và labels về dạng list
train_dataset_dict = process_string_to_array(train_dataset_dict)
dev_dataset_dict = process_string_to_array(dev_dataset_dict)

# Chuyển sang Hugging Face Dataset
train_dataset = HFDataset.from_dict(train_dataset_dict)
dev_dataset = HFDataset.from_dict(dev_dataset_dict)

Processing file: /kaggle/input/vlsp2021ner/vlspner2021/Train-Conll/giaitri_0072.conll
Processing file: /kaggle/input/vlsp2021ner/vlspner2021/Train-Conll/giaitri_0002.conll
Processing file: /kaggle/input/vlsp2021ner/vlspner2021/Train-Conll/kinhte_0059.conll
Processing file: /kaggle/input/vlsp2021ner/vlspner2021/Train-Conll/thegioi_0073.conll
Processing file: /kaggle/input/vlsp2021ner/vlspner2021/Train-Conll/00_add_0047.conll
Processing file: /kaggle/input/vlsp2021ner/vlspner2021/Train-Conll/doisong_0067.conll
Processing file: /kaggle/input/vlsp2021ner/vlspner2021/Train-Conll/doisong_0027.conll
Processing file: /kaggle/input/vlsp2021ner/vlspner2021/Train-Conll/doisong_0013.conll
Processing file: /kaggle/input/vlsp2021ner/vlspner2021/Train-Conll/thethao_0005.conll
Processing file: /kaggle/input/vlsp2021ner/vlspner2021/Train-Conll/vanhoa_0050.conll
Processing file: /kaggle/input/vlsp2021ner/vlspner2021/Train-Conll/00_add_0233.conll
Processing file: /kaggle/input/vlsp2021ner/vlspner2021/Tra

In [4]:
class Example:
    def __init__(self, words, slot_labels, guid=None):
        self.words = words
        self.slot_labels = slot_labels
        self.guid = guid

def convert_to_examples(dataset):
    return [
        Example(words=tokens, slot_labels=labels, guid=i)
        for i, (tokens, labels) in enumerate(zip(dataset['tokens'], dataset['labels']))
    ]

train_examples = convert_to_examples(train_dataset)
dev_examples = convert_to_examples(dev_dataset)

In [5]:
train_unique_labels = extract_labels(train_labels)
dev_unique_labels   = extract_labels(dev_labels)

label_list = sorted(train_unique_labels.union(dev_unique_labels))
print("Labels in train set:", train_unique_labels)
print("Labels in dev set:", dev_unique_labels)
print("Combined label list:", label_list)

label_map = {label: i for i, label in enumerate(label_list)}
num_labels = len(label_list)

Labels in train set: {'I-DATETIME-DATERANGE', 'I-EVENT-GAMESHOW', 'I-DATETIME-DATE', 'B-LOCATION-GPE', 'B-QUANTITY-AGE', 'I-ADDRESS', 'B-ORGANIZATION-STOCK', 'I-ORGANIZATION-SPORTS', 'I-SKILL', 'B-URL', 'I-PRODUCT-AWARD', 'B-PHONENUMBER', 'I-ORGANIZATION-MED', 'I-QUANTITY', 'I-QUANTITY-NUM', 'B-QUANTITY-CUR', 'I-EVENT-CUL', 'I-PERSONTYPE', 'B-LOCATION-GEO', 'I-MISCELLANEOUS', 'I-EVENT-SPORT', 'I-QUANTITY-AGE', 'I-PERSON', 'B-DATETIME-TIME', 'B-PERSON', 'B-EVENT-GAMESHOW', 'I-QUANTITY-DIM', 'B-EVENT-CUL', 'I-QUANTITY-ORD', 'B-SKILL', 'B-QUANTITY-NUM', 'O', 'I-QUANTITY-PER', 'I-URL', 'B-ORGANIZATION-MED', 'I-LOCATION', 'B-EVENT-SPORT', 'B-DATETIME-DATE', 'I-LOCATION-GEO', 'B-ORGANIZATION', 'B-PRODUCT', 'B-ADDRESS', 'I-QUANTITY-CUR', 'B-PRODUCT-COM', 'I-PHONENUMBER', 'I-DATETIME-TIME', 'I-LOCATION-GPE', 'B-DATETIME-TIMERANGE', 'B-PERSONTYPE', 'I-DATETIME-TIMERANGE', 'I-EVENT', 'B-QUANTITY', 'B-DATETIME-DATERANGE', 'B-PRODUCT-LEGAL', 'B-DATETIME', 'B-DATETIME-SET', 'B-QUANTITY-PER', 'I-DAT

In [6]:
num_labels

83

In [7]:
class InputFeatures(object):
    """
    Mỗi đối tượng InputFeatures sẽ chứa các trường đã được tokenize và
    có độ dài cố định (padded).
    """
    def __init__(self, input_ids, attention_mask, token_type_ids, slot_labels_ids):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.slot_labels_ids = slot_labels_ids

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"


def convert_examples_to_features(
    examples,
    max_seq_len,
    tokenizer,
    pad_label_id=-100,  # label này dùng để đánh dấu các vị trí sub-token
    cls_token_segment_id=0,
    pad_token_segment_id=0,
    sequence_segment_id=0,
    mask_padding_with_zero=True,
):
    """
    - examples: danh sách Example (trong đó words và slot_labels là list).
    - max_seq_len: độ dài tối đa cho mỗi câu (sau khi tokenize).
    - tokenizer: tokenizer PhoBERT đã load.
    - pad_label_id: giá trị dùng để padding cho các subword hoặc padding token.
    """

    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    unk_token = tokenizer.unk_token
    pad_token_id = tokenizer.pad_token_id

    features = []

    for example_index, example in enumerate(examples):
        # (Tùy chọn) log tiến trình
        if example_index % 400 == 0:
            logging.info(f"Processing example {example_index} of {len(examples)}")

        tokens = []
        label_ids = []

        # Tokenize từng từ và ánh xạ nhãn
        for word, label in zip(example.words, example.slot_labels):
            word_tokens = tokenizer.tokenize(word)

            if not word_tokens:
                word_tokens = [unk_token]

            tokens.extend(word_tokens)

            # label_map[label] để chuyển tên nhãn -> id
            label_id = label_map[label]
            # nếu 1 từ tách thành nhiều sub-token => sub-token còn lại đánh dấu pad_label_id
            label_ids.extend([label_id] + [pad_label_id] * (len(word_tokens) - 1))

        # cắt chuỗi nếu dài hơn max_seq_len - 2 (chừa chỗ cho [CLS], [SEP])
        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[: (max_seq_len - special_tokens_count)]
            label_ids = label_ids[: (max_seq_len - special_tokens_count)]

        # Thêm [SEP] ở cuối câu
        tokens.append(sep_token)
        label_ids.append(pad_label_id)
        token_type_ids = [sequence_segment_id] * len(tokens)

        # Thêm [CLS] ở đầu câu
        tokens = [cls_token] + tokens
        label_ids = [pad_label_id] + label_ids
        token_type_ids = [cls_token_segment_id] + token_type_ids

        # Chuyển tokens thành input_ids
        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # attention_mask
        attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

        # padding để đủ max_seq_len
        padding_length = max_seq_len - len(input_ids)
        input_ids += [pad_token_id] * padding_length
        attention_mask += [0 if mask_padding_with_zero else 1] * padding_length
        token_type_ids += [pad_token_segment_id] * padding_length
        label_ids += [pad_label_id] * padding_length

        # Tạo đối tượng InputFeatures
        features.append(
            InputFeatures(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                slot_labels_ids=label_ids,
            )
        )

    return features

In [8]:
phobert_model_name = "vinai/phobert-large"

tokenizer = AutoTokenizer.from_pretrained(
    phobert_model_name,
    use_fast=False  # PhoBERT chưa hỗ trợ fast tokenizer
)

model = AutoModelForTokenClassification.from_pretrained(
    phobert_model_name,
    num_labels=num_labels
)

config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.48G [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
class NERDataset(Dataset):
    def __init__(self, features):
        self.features = features

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        return {
            'input_ids': torch.tensor(feature.input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(feature.attention_mask, dtype=torch.long),
            'token_type_ids': torch.tensor(feature.token_type_ids, dtype=torch.long),
            'labels': torch.tensor(feature.slot_labels_ids, dtype=torch.long),
        }

max_seq_len = 128
train_features = convert_examples_to_features(train_examples, max_seq_len, tokenizer)
dev_features = convert_examples_to_features(dev_examples, max_seq_len, tokenizer)

train_dataset_for_torch = NERDataset(train_features)
dev_dataset_for_torch   = NERDataset(dev_features)

In [10]:
def compute_metrics(p: EvalPrediction):
    """
    Hàm tính toán precision, recall, f1 cho toàn bộ nhãn (kể cả 'O'),
    bằng cách flatten danh sách nhãn và dùng classification_report của sklearn.
    """
    predictions = p.predictions.argmax(axis=2)
    labels = p.label_ids

    pred_labels_flat = []
    true_labels_flat = []

    for pred_seq, true_seq in zip(predictions, labels):
        for pred_id, true_id in zip(pred_seq, true_seq):
            if true_id == -100:
                # -100 là pad_label_id, ta bỏ qua
                continue
            pred_labels_flat.append(label_list[pred_id])
            true_labels_flat.append(label_list[true_id])

    report = classification_report(
        true_labels_flat,
        pred_labels_flat,
        labels=label_list,
        zero_division=0,
        output_dict=True  # trả về dict thay vì chuỗi
    )

    print(classification_report(
        true_labels_flat,
        pred_labels_flat,
        labels=label_list,
        zero_division=0
    ))

    f1 = report['micro avg']['f1-score']
    precision = report['micro avg']['precision']
    recall = report['micro avg']['recall']

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [11]:
import wandb
wandb.login(key="29ca57c2758693a1ef4ae2fb01c6e290b7fe70b9")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: nguyentiendat3_t66 (nguyentiendat3_t66-Hanoi University of Science). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_for_torch,
    eval_dataset=dev_dataset_for_torch,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241222_105044-enaj5td1
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ./results
wandb: ⭐️ View project at https://wandb.ai/nguyentiendat3_t66-Hanoi%20University%20of%20Science/huggingface
wandb: 🚀 View run at https://wandb.ai/nguyentiendat3_t66-Hanoi%20University%20of%20Science/huggingface/runs/enaj5td1


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.290900,0.262649,0.929090,0.929090,0.929090
2,0.167400,0.254663,0.932829,0.932829,0.932829
3,0.124300,0.261983,0.932563,0.932563,0.932563


                       precision    recall  f1-score   support

            B-ADDRESS       0.00      0.00      0.00         4
           B-DATETIME       0.62      0.70      0.66      1130
      B-DATETIME-DATE       0.66      0.47      0.55       352
 B-DATETIME-DATERANGE       0.00      0.00      0.00       107
  B-DATETIME-DURATION       0.72      0.71      0.72       420
       B-DATETIME-SET       0.00      0.00      0.00        16
      B-DATETIME-TIME       0.22      0.81      0.35        37
 B-DATETIME-TIMERANGE       0.00      0.00      0.00        78
              B-EMAIL       0.00      0.00      0.00         1
              B-EVENT       0.33      0.13      0.19       106
          B-EVENT-CUL       1.00      0.05      0.10        55
     B-EVENT-GAMESHOW       0.48      0.60      0.54        48
      B-EVENT-NATURAL       1.00      0.18      0.30        17
        B-EVENT-SPORT       0.17      0.88      0.28        43
                 B-IP       0.00      0.00      0.00  

TrainOutput(global_step=2418, training_loss=0.2623530306581429, metrics={'train_runtime': 2067.855, 'train_samples_per_second': 18.696, 'train_steps_per_second': 1.169, 'total_flos': 8978650404740352.0, 'train_loss': 0.2623530306581429, 'epoch': 3.0})

In [13]:
###################################
# 1. Đọc dữ liệu Test từ thư mục .conll
###################################

test_directory = '/kaggle/input/vlsp2021ner/Test-Conll/Test-Conll'
test_sentences, test_labels = read_conll_directory(test_directory)

###################################
# 2. Chuẩn bị dữ liệu Test thành Dataset
###################################
test_dataset_dict = {
    'tokens': test_sentences,
    'labels': test_labels
}

# Chuyển sang dạng list
test_dataset_dict = {
    'tokens': [sentence.split() for sentence in test_dataset_dict['tokens']],
    'labels': [label_seq.split() for label_seq in test_dataset_dict['labels']]
}

# Tạo Hugging Face Dataset
test_dataset_hf = HFDataset.from_dict(test_dataset_dict)

# Chuyển sang Example
test_examples = convert_to_examples(test_dataset_hf)

# Tokenize + Pad tương tự như train/dev
max_seq_len = 128 
test_features = convert_examples_to_features(
    examples=test_examples,
    max_seq_len=max_seq_len,
    tokenizer=tokenizer
)

###################################
# 3. Tạo PyTorch Dataset để evaluate
###################################
class NERDataset(Dataset):
    def __init__(self, features):
        self.features = features

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        f = self.features[idx]
        return {
            'input_ids': torch.tensor(f.input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(f.attention_mask, dtype=torch.long),
            'token_type_ids': torch.tensor(f.token_type_ids, dtype=torch.long),
            'labels': torch.tensor(f.slot_labels_ids, dtype=torch.long),
        }

test_dataset_for_torch = NERDataset(test_features)

Processing file: /kaggle/input/vlsp2021ner/Test-Conll/Test-Conll/1_test_vnexpress_phapluat_0006.conll
Processing file: /kaggle/input/vlsp2021ner/Test-Conll/Test-Conll/0_test_vnexpress_giaoduc_0003.conll
Processing file: /kaggle/input/vlsp2021ner/Test-Conll/Test-Conll/0_test_vnexpress_phapluat_0002.conll
Processing file: /kaggle/input/vlsp2021ner/Test-Conll/Test-Conll/2_test_dantri_batdongsan_0003.conll
Processing file: /kaggle/input/vlsp2021ner/Test-Conll/Test-Conll/2_test_zingnews_giaitri_0005.conll
Processing file: /kaggle/input/vlsp2021ner/Test-Conll/Test-Conll/1_test_vnexpress_phapluat_0003.conll
Processing file: /kaggle/input/vlsp2021ner/Test-Conll/Test-Conll/2_test_zingnews_xahoi_0003.conll
Processing file: /kaggle/input/vlsp2021ner/Test-Conll/Test-Conll/2_test_baomoi_giaoduc_0002.conll
Processing file: /kaggle/input/vlsp2021ner/Test-Conll/Test-Conll/2_test_baomoi_xahoi_0003.conll
Processing file: /kaggle/input/vlsp2021ner/Test-Conll/Test-Conll/2_test_dantri_suckhoe_0001.conll
Pr

In [14]:
print("========= Đánh giá trên tập Test =========")
metrics = trainer.evaluate(test_dataset_for_torch)
print(metrics)

========= Đánh giá trên tập Test =========


                       precision    recall  f1-score   support

            B-ADDRESS       0.00      0.00      0.00        23
           B-DATETIME       0.49      0.64      0.56       618
      B-DATETIME-DATE       0.82      0.58      0.68       576
 B-DATETIME-DATERANGE       0.62      0.21      0.32       140
  B-DATETIME-DURATION       0.86      0.71      0.77       485
       B-DATETIME-SET       0.00      0.00      0.00         4
      B-DATETIME-TIME       0.69      0.93      0.79        54
 B-DATETIME-TIMERANGE       0.86      0.48      0.61       132
              B-EMAIL       0.00      0.00      0.00         2
              B-EVENT       0.46      0.23      0.30       181
          B-EVENT-CUL       0.62      0.67      0.65        15
     B-EVENT-GAMESHOW       0.49      0.61      0.54        57
      B-EVENT-NATURAL       0.25      0.22      0.24         9
        B-EVENT-SPORT       0.61      0.68      0.65       151
                 B-IP       1.00      0.58      0.74  

In [15]:
###############################################################################
# CODE THÊM ĐỂ TEST THỬ 1 ĐOẠN VĂN TÙY Ý
###############################################################################

test_text = """Sofascore là hãng thống kê hàng đầu thế giới, do người Croatia sáng lập năm 2010 
và có hàng chục triệu người dùng. Theo mô hình của hãng, Xuân Son nhận điểm 10 và 
dĩ nhiên là cầu thủ hay nhất trận. Vĩ Hào đứng thứ hai với 8,4 điểm, nhờ một bàn 
và một đường kiến tạo, còn Nguyễn Quang Hải nhận 8,2 điểm với một bàn."""

tokens = test_text.split()
fake_labels = ["O"] * len(tokens)

example_for_infer = Example(
    words=tokens,
    slot_labels=fake_labels,
    guid="inference_1"
)
examples_for_infer = [example_for_infer]

features_for_infer = convert_examples_to_features(
    examples=examples_for_infer,
    max_seq_len=128,  
    tokenizer=tokenizer
)

infer_dataset = NERDataset(features_for_infer)

inference_output = trainer.predict(infer_dataset)
pred_logits = inference_output.predictions
pred_ids = pred_logits.argmax(axis=2)

pred_labels = []
for i, (logits_i, feature_i) in enumerate(zip(pred_ids, features_for_infer)):
    real_token_label_pairs = []
    for idx_token, label_id in enumerate(feature_i.slot_labels_ids):
        if label_id == -100:
            continue
        
        input_id = feature_i.input_ids[idx_token]
        predicted_id = logits_i[idx_token]
        
        # Bỏ qua [CLS], [SEP], [PAD]
        if input_id not in [tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id]:
            subword_str = tokenizer.convert_ids_to_tokens([input_id])[0]
            predicted_label_str = label_list[predicted_id]
            real_token_label_pairs.append((subword_str, predicted_label_str))
    pred_labels.append(real_token_label_pairs)

                       precision    recall  f1-score   support

            B-ADDRESS       0.00      0.00      0.00         0
           B-DATETIME       0.00      0.00      0.00         0
      B-DATETIME-DATE       0.00      0.00      0.00         0
 B-DATETIME-DATERANGE       0.00      0.00      0.00         0
  B-DATETIME-DURATION       0.00      0.00      0.00         0
       B-DATETIME-SET       0.00      0.00      0.00         0
      B-DATETIME-TIME       0.00      0.00      0.00         0
 B-DATETIME-TIMERANGE       0.00      0.00      0.00         0
              B-EMAIL       0.00      0.00      0.00         0
              B-EVENT       0.00      0.00      0.00         0
          B-EVENT-CUL       0.00      0.00      0.00         0
     B-EVENT-GAMESHOW       0.00      0.00      0.00         0
      B-EVENT-NATURAL       0.00      0.00      0.00         0
        B-EVENT-SPORT       0.00      0.00      0.00         0
                 B-IP       0.00      0.00      0.00  

In [16]:
# 8. In kết quả ra 2 cột [Token - NER]
print("===== BẢNG DỰ ĐOÁN NER CHO ĐOẠN VĂN =====")
for token, ner_label in pred_labels[0]:
    print(f"{token}\t{ner_label}")

===== BẢNG DỰ ĐOÁN NER CHO ĐOẠN VĂN =====
Sof@@	B-ORGANIZATION
là	O
hãng	O
th@@	O
kê	O
hàng	O
đầu	O
thế	O
giớ@@	O
do	O
người	B-MISCELLANEOUS
Croatia	I-MISCELLANEOUS
sáng	O
lập	O
năm	B-DATETIME
2010	I-DATETIME
và	O
có	O
hàng	O
chục	O
triệu	O
người	O
dù@@	O
Theo	O
mô	O
hình	O
của	O
hã@@	O
Xuân	B-PERSON
Son	I-PERSON
nhận	O
điểm	O
10	B-QUANTITY-NUM
và	O
dĩ	O
nhiên	O
là	O
cầu	O
thủ	O
hay	O
nhất	O
trậ@@	O
Vĩ	B-PERSON
Hào	I-PERSON
đứng	O
thứ	O
hai	O
với	O
8,4	B-QUANTITY-NUM
điể@@	O
nhờ	O
một	B-QUANTITY-NUM
bàn	O
và	O
một	B-QUANTITY-NUM
đường	O
kiến	O
tạ@@	O
còn	O
Nguyễn	B-PERSON
Quang	I-PERSON
Hải	I-PERSON
nhận	O
8,2	B-QUANTITY-NUM
điểm	O
với	O
một	B-QUANTITY-NUM
bà@@	O
